# Quick Demo: 단일 이미지 복원

데이터셋 없이 단일 이미지로 모델을 테스트합니다.

In [ ]:
import sys
sys.path.insert(0, '..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import torchvision.transforms as T

from src.models import CrossDomainDegradationTransfer

%matplotlib inline

## 1. 모델 로드

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# 모델 로드 (체크포인트가 없으면 랜덤 초기화)
model = CrossDomainDegradationTransfer().to(device)

checkpoint_path = Path('../experiments/best.pth')
if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✓ 체크포인트 로드: {checkpoint_path}")
else:
    print("⚠️ 체크포인트 없음. 랜덤 초기화 모델 사용.")

model.eval()
print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

## 2. 합성 열화 이미지 생성 및 복원

In [ ]:
def add_gaussian_noise(image, sigma=0.1):
    """가우시안 노이즈 추가"""
    noise = torch.randn_like(image) * sigma
    return (image + noise).clamp(-1, 1)

def add_blur(image, kernel_size=5):
    """가우시안 블러 추가"""
    import torch.nn.functional as F
    
    # 간단한 박스 블러
    kernel = torch.ones(1, 1, kernel_size, kernel_size) / (kernel_size ** 2)
    kernel = kernel.to(image.device)
    
    # 각 채널에 적용
    blurred = []
    for c in range(image.shape[1]):
        ch = image[:, c:c+1, :, :]
        ch_blur = F.conv2d(ch, kernel, padding=kernel_size//2)
        blurred.append(ch_blur)
    
    return torch.cat(blurred, dim=1)

# 테스트용 이미지 생성 (체커보드 패턴)
def create_test_image(size=256):
    """테스트용 체커보드 이미지 생성"""
    img = np.zeros((size, size, 3), dtype=np.float32)
    
    # 체커보드 패턴
    block_size = size // 8
    for i in range(8):
        for j in range(8):
            if (i + j) % 2 == 0:
                img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = [0.9, 0.9, 0.9]
            else:
                img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = [0.2, 0.4, 0.6]
    
    # 원 추가
    y, x = np.ogrid[:size, :size]
    center = size // 2
    mask = (x - center)**2 + (y - center)**2 < (size//4)**2
    img[mask] = [0.8, 0.3, 0.3]
    
    return img

In [ ]:
# 테스트 이미지 생성
clean_np = create_test_image(256)

# Tensor로 변환 ([-1, 1] 범위)
clean = torch.from_numpy(clean_np).permute(2, 0, 1).float() * 2 - 1
clean = clean.unsqueeze(0).to(device)

# 열화 적용
degraded_noise = add_gaussian_noise(clean, sigma=0.3)
degraded_blur = add_blur(clean, kernel_size=7)
degraded_both = add_gaussian_noise(add_blur(clean, kernel_size=5), sigma=0.2)

# 복원
with torch.no_grad():
    restored_noise = model.restore(degraded_noise)
    restored_blur = model.restore(degraded_blur)
    restored_both = model.restore(degraded_both)

In [ ]:
# 시각화
def to_numpy(tensor):
    return ((tensor.squeeze(0).cpu().permute(1, 2, 0).numpy() + 1) / 2).clip(0, 1)

fig, axes = plt.subplots(3, 3, figsize=(12, 12))

# Row 1: Gaussian Noise
axes[0, 0].imshow(to_numpy(degraded_noise))
axes[0, 0].set_title('Degraded (Noise σ=0.3)')
axes[0, 0].axis('off')

axes[0, 1].imshow(to_numpy(restored_noise))
axes[0, 1].set_title('Restored')
axes[0, 1].axis('off')

axes[0, 2].imshow(clean_np)
axes[0, 2].set_title('Ground Truth')
axes[0, 2].axis('off')

# Row 2: Blur
axes[1, 0].imshow(to_numpy(degraded_blur))
axes[1, 0].set_title('Degraded (Blur k=7)')
axes[1, 0].axis('off')

axes[1, 1].imshow(to_numpy(restored_blur))
axes[1, 1].set_title('Restored')
axes[1, 1].axis('off')

axes[1, 2].imshow(clean_np)
axes[1, 2].set_title('Ground Truth')
axes[1, 2].axis('off')

# Row 3: Both
axes[2, 0].imshow(to_numpy(degraded_both))
axes[2, 0].set_title('Degraded (Blur + Noise)')
axes[2, 0].axis('off')

axes[2, 1].imshow(to_numpy(restored_both))
axes[2, 1].set_title('Restored')
axes[2, 1].axis('off')

axes[2, 2].imshow(clean_np)
axes[2, 2].set_title('Ground Truth')
axes[2, 2].axis('off')

plt.suptitle('Synthetic Degradation Test', fontsize=14)
plt.tight_layout()
plt.show()

## 3. 사용자 이미지로 테스트

자신의 이미지로 테스트하려면 아래 셀의 `IMAGE_PATH`를 수정하세요.

In [ ]:
# 이미지 경로 설정 (수정 필요)
IMAGE_PATH = None  # 예: 'examples/my_image.jpg'

if IMAGE_PATH and Path(IMAGE_PATH).exists():
    # 이미지 로드 및 전처리
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    
    img = Image.open(IMAGE_PATH).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    # 복원
    with torch.no_grad():
        restored = model.restore(img_tensor)
    
    # 시각화
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    axes[0].imshow(to_numpy(img_tensor))
    axes[0].set_title('Input')
    axes[0].axis('off')
    
    axes[1].imshow(to_numpy(restored))
    axes[1].set_title('Restored')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("이미지 경로를 설정하세요. 예: IMAGE_PATH = 'examples/my_image.jpg'")

## 4. 모델 구조 확인

In [ ]:
print("모델 구조:")
print("="*60)
print(model)

In [ ]:
# 각 컴포넌트별 파라미터 수
components = {
    'DegradationEncoder': model.deg_encoder,
    'ContentEncoder': model.content_encoder,
    'Decoder': model.decoder,
    'DomainDiscriminator': model.domain_disc,
}

print("\n컴포넌트별 파라미터 수:")
print("="*40)
total = 0
for name, module in components.items():
    n_params = sum(p.numel() for p in module.parameters())
    total += n_params
    print(f"{name:<25} {n_params:>10,}")
print("="*40)
print(f"{'Total':<25} {total:>10,}")